In [ ]:
!pip install langgraph

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal

class RoutingState(TypedDict):
    query: str
    query_type: str
    response: str
    handled_by: str

def classify_query(state: RoutingState) -> dict:
    query = state["query"].lower()
    print(f"Received query: {state['query']}")

    if any(k in query for k in ["salary", "pay", "compensation"]):
        query_type = "compensation"
    elif any(k in query for k in ["benefit", "health insurance", "401k"]):
        query_type = "benefits"
    elif any(k in query for k in ["leave", "vacation", "pto"]):
        query_type = "leave_policy"
    else:
        query_type = "general"

    print(f"Classified as: {query_type}")
    return {"query_type": query_type}

def handle_compensation(state: RoutingState) -> dict:
    print("Routing to Compensation Team...")
    return {"response": "Salary info is in your portal.", "handled_by": "Compensation Team"}

def handle_benefits(state: RoutingState) -> dict:
    print("Routing to Benefits Team...")
    return {"response": "Details are in your benefits booklet.", "handled_by": "Benefits Team"}

def handle_leave_policy(state: RoutingState) -> dict:
    print("Routing to HR Policies...")
    return {"response": "You have 15 days PTO per year.", "handled_by": "HR Policies"}

def handle_general(state: RoutingState) -> dict:
    print("Routing to General HR...")
    return {"response": "Please contact HR@company.com.", "handled_by": "General HR"}

def route_query(state: RoutingState) -> str:
    return state["query_type"]

workflow = StateGraph(RoutingState)
workflow.add_node("classify", classify_query)
workflow.add_node("compensation", handle_compensation)
workflow.add_node("benefits", handle_benefits)
workflow.add_node("leave_policy", handle_leave_policy)
workflow.add_node("general", handle_general)

workflow.set_entry_point("classify")
workflow.add_conditional_edges(
    "classify",
    route_query,
    {
        "compensation": "compensation",
        "benefits": "benefits",
        "leave_policy": "leave_policy",
        "general": "general"
    }
)

workflow.add_edge("compensation", END)
workflow.add_edge("benefits", END)
workflow.add_edge("leave_policy", END)
workflow.add_edge("general", END)

app = workflow.compile()

test_queries = [
    "What is my current salary?",
    "How do I apply for vacation?",
    "What are the health insurance options?",
    "Where is the coffee machine?"
]

for query in test_queries:
    print("\n" + "="*60)
    result = app.invoke({"query": query})
    print(f"Response: {result['response']}")
    print(f"Handled by: {result['handled_by']}")


Received query: What is my current salary?
Classified as: compensation
Routing to Compensation Team...
Response: Salary info is in your portal.
Handled by: Compensation Team

Received query: How do I apply for vacation?
Classified as: leave_policy
Routing to HR Policies...
Response: You have 15 days PTO per year.
Handled by: HR Policies

Received query: What are the health insurance options?
Classified as: benefits
Routing to Benefits Team...
Response: Details are in your benefits booklet.
Handled by: Benefits Team

Received query: Where is the coffee machine?
Classified as: general
Routing to General HR...
Response: Please contact HR@company.com.
Handled by: General HR
